In [1]:
!pip install xgboost==1.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.5/173.5 MB 6.9 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.0.3
    Uninstalling xgboost-2.0.3:
      Successfully uninstalled xgboost-2.0.3


In [1]:
# 설치 후 런타임 - 세션 다시 시작

In [2]:
import xgboost as xgb
print(xgb.__version__)

/usr/local/lib/python3.10/dist-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


1.5.0


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from sklearn.datasets import load_breast_cancer
import pandas as pd

cancer = load_breast_cancer()

data_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
data_df['target'] = cancer.target
data_df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [5]:
# 데이터 분할

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data_df.drop("target", axis = 1),
    data_df['target'],
    random_state=42
)

# train-valid 분할
# valid 로 검증

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, random_state=42
)

In [7]:
# DMatrix 변환
#- 넘파이 배열, 판다스 데이터프레임에서도 변환이 가능
#-python wrapper 에서 사용하는 데이터 형식

In [8]:
d_train = xgb.DMatrix(data=X_train, label=y_train)
d_valid = xgb.DMatrix(data=X_valid, label= y_valid)
d_test = xgb.DMatrix(data=X_test, label=y_test)

In [9]:
#하이퍼 파라미터 설정

In [18]:
params = {
    'max_depth': 3, # 각 나무의 최대 깊이
    'eta': 0.05, # 학습률(learning rate)
    'objective': 'binary:logistic', # 목적 함수(모델이 예측의 결과를 어떻게 표현할지를 지정하는 함수). logistic 함수를 사용하기 때문에 확률로 결과물을 내어준다.
    'eval_metric': 'logloss' # Error(성능) 측정 함수
}

# 학습 횟수
num_rounds = 400

In [11]:
# 학습 및 검증에 사용될 데이터 세트 지정하기
# 'train' : 학습
# 'eval' : 검증

In [17]:
eval_list = [
    (d_train, 'train'),
    (d_valid, 'eval')
]

In [13]:
# 훈련

In [19]:
xgb_model = xgb.train(
    params=params, # 하이퍼 파라미터가 들어있는 딕셔너리
    dtrain = d_train, # 훈련 데이터 Feature + Label
    num_boost_round=10000, # 훈련 횟수
    early_stopping_rounds=50, # 성능 개선(Loss가 낮아지면 성능이 개선됨)이 50라운드 이내에 이루어지지 않으면 학습을 종료
    evals = eval_list # 검증 세트 지정. 여기에서는 훈련 세트와 검증세트에 대한 성능을 한꺼번에 확인
)

[0]	train-logloss:0.65124	eval-logloss:0.65404
[1]	train-logloss:0.61200	eval-logloss:0.62023
[2]	train-logloss:0.57754	eval-logloss:0.58633
[3]	train-logloss:0.54460	eval-logloss:0.55826
[4]	train-logloss:0.51435	eval-logloss:0.53266
[5]	train-logloss:0.48771	eval-logloss:0.50902
[6]	train-logloss:0.46188	eval-logloss:0.48738
[7]	train-logloss:0.43893	eval-logloss:0.46707
[8]	train-logloss:0.41664	eval-logloss:0.44913
[9]	train-logloss:0.39683	eval-logloss:0.43179
[10]	train-logloss:0.37746	eval-logloss:0.41644
[11]	train-logloss:0.36018	eval-logloss:0.40182
[12]	train-logloss:0.34321	eval-logloss:0.38998
[13]	train-logloss:0.32734	eval-logloss:0.37724
[14]	train-logloss:0.31250	eval-logloss:0.36721
[15]	train-logloss:0.29930	eval-logloss:0.35642
[16]	train-logloss:0.28621	eval-logloss:0.34786
[17]	train-logloss:0.27395	eval-logloss:0.34002
[18]	train-logloss:0.26244	eval-logloss:0.33208
[19]	train-logloss:0.25146	eval-logloss:0.32362
[20]	train-logloss:0.24147	eval-logloss:0.31627
[2

In [20]:
# 예측

In [21]:
import numpy as np

# 모델 만들 때 objective 함수를 logistic으로 지정했기 때문에 예측시 확률 형식으로 예측 됨

pred_props = xgb_model.predict(d_test)
print(np.round(pred_props[:10],3)) # 양성(1)로 분류될 확률

[0.935 0.001 0.    0.999 1.    0.001 0.002 0.642 0.521 0.997]


In [22]:
# 0 과 1로 이루어진 결과로 보기
# -0.5를 넘으면 1, 0.5가 안되면 0
preds = [1 if x >0.5 else 0 for x in pred_props]
preds[ :10]

[1, 0, 0, 1, 1, 0, 0, 1, 1, 1]

In [23]:
####### XGBoost = Scikit Learn Wrapper #######

In [24]:
from xgboost import XGBClassifier

# 조기종료 안 함
xgb_clf = XGBClassifier(
    n_estimators= 400, # 학습할 학습기의 개수(훈련 횟수)
    learning_rate = 0.05, # 학습률
    max_depth = 3,
    eval_metric= 'logloss'
)

In [25]:
# 훈련
xgb_clf.fit(X_train, y_train, verbose=True) # verbose : 로그 출력 여부

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metric='logloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.05, max_delta_step=0,
              max_depth=3, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=400, n_jobs=2,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [30]:
# 사이킷런의 predict : 결정된 클래스가 나옴
preds = xgb_clf.predict(X_test)
preds[:10]

array([1, 0, 0, 1, 1, 0, 0, 1, 0, 1])

In [32]:
# predict_proba를 이용해 각 예측된 클래스의 확률 정보 확인 가능
pred_proba = xgb_clf.predict_proba(X_test)
pred_proba[:10]

array([[7.0421875e-02, 9.2957813e-01],
       [9.9837631e-01, 1.6236610e-03],
       [9.9944514e-01, 5.5486610e-04],
       [1.5223622e-03, 9.9847764e-01],
       [2.8401613e-04, 9.9971598e-01],
       [9.9914682e-01, 8.5319200e-04],
       [9.9682111e-01, 3.1789127e-03],
       [4.3057954e-01, 5.6942046e-01],
       [5.5072081e-01, 4.4927916e-01],
       [3.7894249e-03, 9.9621058e-01]], dtype=float32)

In [35]:
# early stopping
xgb_clf = XGBClassifier(
    n_estimators=1000,
    learing_rate=0.05,
    max_depth=3,
    eval_metrics='logloss'
)

In [34]:
# 검증 세트를 따로 지정

# eval_sets : 검증 데이터 목록
# train valid : 훈련 세트로 검증을 하고, 검증세트로 검증
# 훈련 세트로 검증 : 학습한 내용에대한 검증을 수행
# 검증 세트로 검증 : 모르는 데이터에 대한 검증을 수행

# 두 개를 같이 하는 이유 : 과대 적합인지 확인하기 위해

eval_sets = [
    (X_train, y_train),
    (X_valid, y_valid) # 더이상 수정을 하지 않겠다 = 튜플
]

xgb_clf.fit(
    X_train, y_train,
    early_stopping_rounds=50,
    eval_set = eval_sets,
    verbose=True
)
      # 훈련 테스트 로스                # 검증 테스트 로스

[01:38:22] WARNING: ../src/learner.cc:576: 
Parameters: { "eval_metrics", "learing_rate" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[01:38:22] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	validation_0-logloss:0.47223	validation_1-logloss:0.48805
[1]	validation_0-logloss:0.34045	validation_1-logloss:0.37900
[2]	validation_0-logloss:0.25744	validation_1-logloss:0.31620
[3]	validation_0-logloss:0.19984	validation_1-logloss:0.27947
[4]	validation_0-logloss:0.15985	validation_1-logloss:0.24494
[5]	validation_0-logloss:0.13100	validation_1-logloss:0.230

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              eval_metrics='logloss', gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learing_rate=0.05,
              learning_rate=0.300000012, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=400, n_jobs=2, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)